# WhatsApp

本 Notebook 展示了如何使用 WhatsApp 聊天加载器。此类有助于将导出的 WhatsApp 对话映射到 LangChain 聊天消息。

该过程包括三个步骤：
1. 将聊天对话导出到计算机
2. 创建 `WhatsAppChatLoader`，并将文件路径指向 JSON 文件或 JSON 文件目录
3. 调用 `loader.load()`（或 `loader.lazy_load()`）执行转换。

## 1. 创建消息转储

要导出您的 WhatsApp 对话，请完成以下步骤：

1. 打开目标对话
2. 点击右上角的三个点，然后选择“更多”。
3. 然后选择“导出聊天”并选择“不含媒体”。

下面是每个对话数据格式的示例：

In [1]:
%%writefile whatsapp_chat.txt
[8/15/23, 9:12:33 AM] Dr. Feather: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.
[8/15/23, 9:12:43 AM] Dr. Feather: I spotted a rare Hyacinth Macaw yesterday in the Amazon Rainforest. Such a magnificent creature!
‎[8/15/23, 9:12:48 AM] Dr. Feather: ‎image omitted
[8/15/23, 9:13:15 AM] Jungle Jane: That's stunning! Were you able to observe its behavior?
‎[8/15/23, 9:13:23 AM] Dr. Feather: ‎image omitted
[8/15/23, 9:14:02 AM] Dr. Feather: Yes, it seemed quite social with other macaws. They're known for their playful nature.
[8/15/23, 9:14:15 AM] Jungle Jane: How's the research going on parrot communication?
‎[8/15/23, 9:14:30 AM] Dr. Feather: ‎image omitted
[8/15/23, 9:14:50 AM] Dr. Feather: It's progressing well. We're learning so much about how they use sound and color to communicate.
[8/15/23, 9:15:10 AM] Jungle Jane: That's fascinating! Can't wait to read your paper on it.
[8/15/23, 9:15:20 AM] Dr. Feather: Thank you! I'll send you a draft soon.
[8/15/23, 9:25:16 PM] Jungle Jane: Looking forward to it! Keep up the great work.

Writing whatsapp_chat.txt


## 2. 创建聊天加载器

WhatsAppChatLoader 接受生成的 zip 文件、解压后的目录，或其中任何 chat `.txt` 文件的路径。

一并提供用户名，您希望在微调时扮演“AI”的角色。

In [7]:
from langchain_community.chat_loaders.whatsapp import WhatsAppChatLoader

In [12]:
loader = WhatsAppChatLoader(
    path="./whatsapp_chat.txt",
)

## 3. 加载消息

`load()` (或 `lazy_load`) 方法会返回一个“ChatSessions”列表，其中存储了当前已加载会话的消息列表。

In [13]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "Dr. Feather" to AI messages
messages: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="Dr. Feather")
)

[{'messages': [AIMessage(content='I spotted a rare Hyacinth Macaw yesterday in the Amazon Rainforest. Such a magnificent creature!', additional_kwargs={'sender': 'Dr. Feather', 'events': [{'message_time': '8/15/23, 9:12:43 AM'}]}, example=False),
   HumanMessage(content="That's stunning! Were you able to observe its behavior?", additional_kwargs={'sender': 'Jungle Jane', 'events': [{'message_time': '8/15/23, 9:13:15 AM'}]}, example=False),
   AIMessage(content="Yes, it seemed quite social with other macaws. They're known for their playful nature.", additional_kwargs={'sender': 'Dr. Feather', 'events': [{'message_time': '8/15/23, 9:14:02 AM'}]}, example=False),
   HumanMessage(content="How's the research going on parrot communication?", additional_kwargs={'sender': 'Jungle Jane', 'events': [{'message_time': '8/15/23, 9:14:15 AM'}]}, example=False),
   AIMessage(content="It's progressing well. We're learning so much about how they use sound and color to communicate.", additional_kwargs={

### 后续步骤

之后你可以根据自己的需求来使用这些消息，例如对模型进行微调，选取少样本示例，或者直接预测下一条消息。

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[0]["messages"]):
    print(chunk.content, end="", flush=True)

Thank you for the encouragement! I'll do my best to continue studying and sharing fascinating insights about parrot communication.